# 基本面选股

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ

d:\Anaconda\envs\py38\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## 基本面因子

### 盈利性
+ 毛利率 - netprofit_margin
+ 净利率 - grossprofit_margin
+ 扣非净资产收益率 - roe_dt
+ 营收同比增长率 - or_yoy
+ 净利润同比增长率 - dt_netprofit_yoy (归母扣非净利润同比增长率)
+ 扣非净利润 - dtprofit_to_profit

### 流动性
+ 资产负债率 - debt_to_assets
+ 流动比率 - current_ratio
+ 速动比率 - quick_ratio
+ 应收账款周转率 - ar_turn
+ 应收账款账龄分布
+ 存货周转率 - inv_turn

### 估值
+ PE倍数（市盈率）- pe
+ PB倍数（市净率）- pb
+ PEG

### 其他
+ 商誉占总资产比重 goodwill / total_assets
+ 商誉占净资产比重 goodwill / (total_assets - total_liab)

In [5]:
def fund_ind(token, stock_code,
             start=datetime.datetime.now()-datetime.timedelta(days=365),
             end=datetime.datetime.now()):
    ts.set_token(token)
    pro = ts.pro_api()

    ### Finance Indicators
    find_fields = ['netprofit_margin', 'grossprofit_margin', 'roe_dt', 'or_yoy', 'dt_netprofit_yoy', 'dtprofit_to_profit',
                   'debt_to_assets', 'current_ratio', 'quick_ratio', 'ar_turn', 'inv_turn']
    df_find = pro.fina_indicator(ts_code=stock_code, 
                                 start_date=start.strftime('%Y%m%d'),
                                 end_date=end.strftime('%Y%m%d'))
    non_index_list = [x for x in find_fields if x not in df_find.columns]
    df_find = df_find[['end_date'] + [x for x in find_fields if x in df_find.columns]].drop_duplicates()
    for non_index in non_index_list:
        df_find[non_index] = 0
    df_find.rename(columns = {'end_date':'trade_date'}, inplace = True)
    

    ### Balance Sheets
    df_bs = pro.balancesheet(ts_code=stock_code, 
                         start_date=start.strftime('%Y%m%d'),
                         end_date=end.strftime('%Y%m%d'), 
                         fields='end_date, goodwill, total_assets, total_liab')
    df_bs['goodwill'] = df_bs['goodwill'].fillna(0)
    df_bs['gw/a'] = df_bs.goodwill / df_bs.total_assets
    df_bs['gw/n'] = df_bs.goodwill / (df_bs.total_assets - df_bs.total_liab)
    df_bs.drop(columns=['goodwill', 'total_assets', 'total_liab'], inplace=True)
    df_bs.rename(columns={'end_date':'trade_date'}, inplace=True)

    ### Daliy
    df_daily = pro.daily_basic(ts_code=stock_code, 
                           start_date=start.strftime('%Y%m%d'),
                           end_date=end.strftime('%Y%m%d'), 
                           fields='trade_date, pe, pb')[::-1]
    df_daily['pe'] = df_daily['pe'].fillna(0)

    df_final = df_daily.merge(df_find, on='trade_date', how='left').merge(df_bs, on='trade_date', how='left')
    df_final['trade_date'] = pd.to_datetime(df_final.trade_date)
    df_final.index=pd.to_datetime(df_final.trade_date)
    df_final.drop('trade_date', axis=1, inplace = True)
    df_final.fillna(method='bfill', inplace=True)
    df_final.fillna(method='pad', inplace=True)
    return df_final

# 沪深300循环

In [3]:
condition = {
    'netprofit_margin' : (1, 30),
    'grossprofit_margin' : (1, 20),
    'roe_dt' : (1, 15),
    'or_yoy' : (1, 15),
    'dt_netprofit_yoy' : (1, 15),
    'debt_to_assets' : (-1, 50),
    'current_ratio' : (1, 2),
    'quick_ratio' : (1, 2),
    'pe' : (1, 30),
    'pb' : (1, 1)
}

In [6]:
s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 365)
e_date = datetime.date(2020, 12, 31)
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'
stock_index = '000300.SH'
pro = ts.pro_api(token)
stock_list = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code').ts_code

comp_per = {}
for s_code in stock_list:
    try:
        comp_per[s_code] = 0
        df = fund_ind(token, s_code, s_date, e_date)
        if len(df) != 0:
            for col in df.columns:
                if col in condition.keys() and df.loc[:, col][-1] != 0:
                    if condition[col][0] == 1:
                        comp_per[s_code] += 1 if df.loc[:, col][-1] >= condition[col][1] else 0
                    else:
                        comp_per[s_code] += 1 if df.loc[:, col][-1] <= condition[col][1] else 0
            print((s_code, comp_per[s_code]))
            time.sleep(4)
    except:
        print(s_code)

000001.SZ
('000002.SZ', 3)
('000004.SZ', 8)
('000005.SZ', 3)
('000006.SZ', 5)
('000007.SZ', 4)
('000008.SZ', 2)
('000009.SZ', 3)
('000010.SZ', 2)
('000011.SZ', 4)
('000012.SZ', 5)
('000014.SZ', 3)
('000016.SZ', 2)
('000017.SZ', 4)
('000019.SZ', 3)
('000020.SZ', 3)
('000021.SZ', 3)
('000023.SZ', 2)
('000025.SZ', 5)
('000026.SZ', 5)
('000027.SZ', 3)
('000028.SZ', 1)
('000029.SZ', 5)
('000030.SZ', 2)
('000031.SZ', 1)
('000032.SZ', 6)
('000034.SZ', 1)
('000035.SZ', 1)
('000036.SZ', 4)
('000037.SZ', 5)
('000038.SZ', 7)
('000039.SZ', 2)
('000040.SZ', 0)
('000042.SZ', 3)
('000045.SZ', 6)
('000046.SZ', 1)
('000048.SZ', 3)
('000049.SZ', 3)
('000050.SZ', 3)
('000055.SZ', 1)
('000056.SZ', 2)
('000058.SZ', 3)
('000059.SZ', 1)
('000060.SZ', 3)
('000061.SZ', 4)
('000062.SZ', 1)
('000063.SZ', 4)
('000065.SZ', 2)
('000066.SZ', 4)
('000068.SZ', 3)
('000069.SZ', 2)
('000070.SZ', 1)
('000078.SZ', 2)
('000088.SZ', 7)
('000089.SZ', 2)
('000090.SZ', 4)
('000096.SZ', 5)
('000099.SZ', 5)
('000100.SZ', 3)
('00

In [8]:
sorted(comp_per.items(), key=lambda x:x[1], reverse=True)

[('000661.SZ', 10),
 ('002007.SZ', 10),
 ('002214.SZ', 10),
 ('002901.SZ', 10),
 ('002932.SZ', 10),
 ('002978.SZ', 10),
 ('002980.SZ', 10),
 ('002985.SZ', 10),
 ('003026.SZ', 10),
 ('300394.SZ', 10),
 ('300443.SZ', 10),
 ('300529.SZ', 10),
 ('300595.SZ', 10),
 ('300677.SZ', 10),
 ('300699.SZ', 10),
 ('300701.SZ', 10),
 ('300726.SZ', 10),
 ('300760.SZ', 10),
 ('300770.SZ', 10),
 ('300776.SZ', 10),
 ('300777.SZ', 10),
 ('300782.SZ', 10),
 ('300832.SZ', 10),
 ('300841.SZ', 10),
 ('300888.SZ', 10),
 ('300896.SZ', 10),
 ('603258.SH', 10),
 ('603444.SH', 10),
 ('603666.SH', 10),
 ('000416.SZ', 9),
 ('002022.SZ', 9),
 ('002027.SZ', 9),
 ('002030.SZ', 9),
 ('002414.SZ', 9),
 ('002507.SZ', 9),
 ('002568.SZ', 9),
 ('002833.SZ', 9),
 ('002838.SZ', 9),
 ('002859.SZ', 9),
 ('002919.SZ', 9),
 ('002975.SZ', 9),
 ('002979.SZ', 9),
 ('002987.SZ', 9),
 ('003028.SZ', 9),
 ('300142.SZ', 9),
 ('300151.SZ', 9),
 ('300206.SZ', 9),
 ('300379.SZ', 9),
 ('300408.SZ', 9),
 ('300474.SZ', 9),
 ('300481.SZ', 9),
 (

# 全市场循环